Using two CSV files in hand from previous work projects

Import dependencies  

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import username
from config import password

Store CSV, memberData into DataFrame

In [2]:
csv_file = "Resources/memberData.csv"
memberData_df = pd.read_csv(csv_file)
memberData_df.head()

,program,zip,state,member_cnt
0,Outreach,3103,AK,11
1,Outreach,46227,AL,147
2,Outreach,60623,AR,"15,409"
3,Outreach,60629,AZ,"17,009"
4,Outreach,3060,CA,"63,496"


Create new data with needed columns. Will not include program and zip.

In [3]:
new_memberData_df = memberData_df[['state', 'member_cnt']].copy()
new_memberData_df.head()

,state,member_cnt
0,AK,11
1,AL,147
2,AR,"15,409"
3,AZ,"17,009"
4,CA,"63,496"


Clean the data by dropping duplicates and setting the index

In [4]:
new_memberData_df.drop_duplicates("state", inplace = True)
new_memberData_df.set_index("state", inplace = True)
new_memberData_df.head()

,member_cnt
state,
AK,11
AL,147
AR,"15,409"
AZ,"17,009"
CA,"63,496"


Store CSV, txnData into DataFrame

In [5]:
csv_file = "Resources/txnData.csv"
txnData_df = pd.read_csv(csv_file)
txnData_df.head()

,program,state,txn_amt,txn_cnt
0,Outreach,AK,4,2
1,Outreach,AL,133,23
2,Outreach,AR,100,16
3,Outreach,AZ,15323,40
4,Outreach,CA,9983,69


Create new data with needed columns. Will remove program.

In [6]:
new_txnData_df = txnData_df[['state','txn_amt','txn_cnt']].copy()
new_txnData_df.head()

,state,txn_amt,txn_cnt
0,AK,4,2
1,AL,133,23
2,AR,100,16
3,AZ,15323,40
4,CA,9983,69


In [7]:
new_txnData_df.drop_duplicates("state", inplace = True)
new_txnData_df.set_index("state", inplace = True)
new_txnData_df.head()

,txn_amt,txn_cnt
state,,
AK,4,2
AL,133,23
AR,100,16
AZ,15323,40
CA,9983,69


Create "ETLProject_db" in Postgres

Double click this cell for Query instructions - or use the sql query file provided.
________________________________________________________________
Create a table in Postres for each CSV. Member and Transaction
Query:
CREATE TABLE member (
	state TEXT PRIMARY KEY,
	member_cnt TEXT
);

CREATE TABLE transaction (
	state TEXT PRIMARY KEY,
	txn_amt INT,
	txn_cnt INT
);

Connect to local database

In [18]:
rds_connection_string = f"{username}:{password}@localhost:5432/ETLProject_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

Check for tables

In [19]:
engine.table_names()

['member', 'transaction']

Use pandas to load member csv converted DataFrame into database

In [23]:
new_memberData_df.to_sql(name='member', con=engine, if_exists='append', index=True)

Use pandas to load transaction csv converted DataFrame into database

In [ ]:
new_txnData_df.to_sql(name='transaction', con=engine, if_exists='append', index=True)

Confirm data has been added by querying the member table
____________________________________________________________
query in Postgres:
Select * from member*  

In [20]:
pd.read_sql_query('select * from member', con=engine).head()

,state,member_cnt
0,AK,11
1,AL,147
2,AR,"15,409"
3,AZ,"17,009"
4,CA,"63,496"


Confirm data has been added by querying the transaction table
_______________________________________________
query in Postgres:
Select * from transaction*

In [21]:
pd.read_sql_query('select * from transaction', con=engine).head()

,state,txn_amt,txn_cnt
0,AK,4,2
1,AL,133,23
2,AR,100,16
3,AZ,15323,40
4,CA,9983,69


Double click for Query instructions - Join the 2 tables by "state" in Postgres - or use the sql query file provided.
_____________________________________________________________________________________
SELECT member.state, member.member_cnt, transaction.txn_amt, transaction.txn_cnt
FROM member
JOIN transaction
ON member.state = transaction.state;